In [1]:
from pathlib import Path
import joblib
from typing import List

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline

from pipeline import (
    DropHighNAPercentage,
    DropColumns,
    NanInputer,
)
from utils import (
    show,
    show_null_percentages,
)

pd.set_option("display.max_columns", None)

In [2]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"

DATA_PATH = DATA_DIR / 'indices_institucional_2005_2022.xlsx'
DATA_PATH.exists()

True

### Loading Data

In [3]:
# inmuebles = pd.read_excel(DATA_PATH, sheet_name="Inmuebles")
# labs = pd.read_excel(DATA_PATH, sheet_name="Laboratorios y Talleres")
# docentes = pd.read_excel(DATA_PATH, sheet_name="Docentes")

# extra_data = inmuebles, labs, docentes
# joblib.dump(extra_data, DATA_DIR / 'extra_data.pkl')

In [4]:
inmuebles, labs, docentes = joblib.load(DATA_DIR / 'extra_data.pkl')
df = joblib.load(DATA_DIR / 'processed_df.pkl')

### Preprocessing Data

In [5]:
def prepare_for_merge(df_to_merge: pd.DataFrame, year_col: str) -> pd.DataFrame:
    df_to_merge["Año"] = df_to_merge[year_col].astype(int)
    df_to_merge = df_to_merge.drop(columns=[year_col])
    df_to_merge = df_to_merge.rename(columns={"idSede": "Cód. Sede"})
    df_to_merge["Cód. Sede"] = df_to_merge["Cód. Sede"].astype(int)
    df_to_merge["Cód. Institución"] = df_to_merge["Cód. Institución"].astype(int)
    df_to_merge = df_to_merge.drop(columns=["Tipo Institución", "Cód. Institución"])

    return df_to_merge


inmuebles = prepare_for_merge(inmuebles, year_col="Año Información")
labs = prepare_for_merge(labs, year_col="Año Proceso")
docentes = prepare_for_merge(docentes, year_col="Año Proceso")

In [7]:
# pipes
inmuebles_pipe = Pipeline(
    [
        (
            "drop_columns",
            DropColumns(
                columns=["Nombre Institución", "Sede", "idInstitucion"], errors="ignore"
            ),
        ),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.3, exclude=[])),
        ("inputer", NanInputer(n_neighbors=5)),
    ]
)

labs_pipe = Pipeline(
    [
        (
            "drop_columns",
            DropColumns(
                columns=["Nombre Institución", "Sede", "idInstitucion"], errors="ignore"
            ),
        ),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.3, exclude=[])),
        ("inputer", NanInputer(n_neighbors=5)),
    ]
)

docentes_pipe = Pipeline(
    [
        (
            "drop_columns",
            DropColumns(
                columns=["Nombre Institución", "Sede", "idInstitucion"], errors="ignore"
            ),
        ),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.3, exclude=[])),
        ("inputer", NanInputer(n_neighbors=5)),
    ]
)

In [8]:
inmuebles = inmuebles_pipe.fit_transform(inmuebles)
labs = labs_pipe.fit_transform(labs)
docentes = docentes_pipe.fit_transform(docentes)

In [9]:
show(inmuebles)
show(labs)
show(docentes)
show(df)

(5823, 8)


,Cód. Sede,N° Inmuebles,M2 Terreno,M2 Construido,M2 Salas,Nº Oficinas,Nº Salas,Año
0,1001001.0,150.0,109043910.0,705759.0,50986.0,3500.0,708.0,2022.0
1,1001001.0,140.0,109046688.0,692010.0,51599.0,3500.0,736.0,2021.0
2,1001001.0,140.0,109097053.0,681132.0,50716.0,3500.0,707.0,2020.0
3,1001001.0,140.0,109097053.0,680498.0,44829.0,3500.0,768.0,2019.0
4,1001001.0,140.0,108016252.0,675606.0,43285.0,3489.0,775.0,2018.0


(5567, 6)


,Cód. Sede,Nº Laboratorios,M2 Construido,Nº de PC para alumnos,N° Computadores con Internet,Año
0,1001001.0,1091.0,64706.0,5325.0,5231.0,2022.0
1,1001001.0,1103.0,65251.0,5190.0,5089.0,2021.0
2,1001001.0,1103.0,65251.0,4718.0,4624.0,2020.0
3,1001001.0,1012.0,69207.0,4280.0,4177.0,2019.0
4,1001001.0,996.0,68435.0,4248.0,4248.0,2018.0


(5845, 18)


,Cód. Sede,N°DocentesJornadaMedia,N°DocentesJornadaHora,N°DocentesJornadaCompleta,N°HorasJornadaCompleta,N°HorasJornadaMedia,N°HorasJornadaHora,N°HorasProfJornadaHora,N°HorasProfJornadaMedia,N°HorasProfJornadaCompleta,N°ProfesionalJornadaHora,N°ProfesionalJornadaMedia,N°ProfesionalJornadaCompleta,N°Docentes,N°Horas,N°DocentesHombres,N°DocentesMujeres,Año
0,1001001.0,680.0,1584.0,1727.0,75316.0,15040.0,14201.0,4117.0,4299.0,8129.0,464.0,194.0,190.0,3991.0,104557.0,2440.0,1551.0,2022.0
1,1001001.0,708.0,1544.0,1739.0,75838.0,15663.0,13929.0,4523.0,4771.0,8643.0,508.0,215.0,202.0,3991.0,105430.0,2454.0,1537.0,2021.0
2,1001001.0,684.0,1496.0,1762.0,76737.0,15169.0,13609.0,5710.0,5872.0,11418.0,617.0,265.0,268.0,3942.0,105515.0,2441.0,1501.0,2020.0
3,1001001.0,688.0,1491.0,1779.0,77630.0,15293.0,13301.0,5665.0,6180.0,11812.0,632.0,278.0,276.0,3958.0,106224.0,2481.0,1477.0,2019.0
4,1001001.0,692.0,1420.0,1744.0,76140.0,15383.0,12818.0,5393.0,5927.0,12802.0,583.0,267.0,296.0,3856.0,104341.0,2446.0,1410.0,2018.0


(39726, 161)


,Año,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,Tipo Programa,Tipo Carrera,IngresoDirecto,Año Inicio Actividades,Duración (en semestres),Cód. Sede,Promedio Puntaje (promedio matemáticas y lenguaje),Puntaje de corte (primer seleccionado),Puntaje de corte (promedio de la carrera),Puntaje de corte (último seleccionado),Nº Alumnos Ingreso Via PSU o PDT,Valor de matrícula,Valor de arancel,Valor del Título,Vacantes,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula Total,Nombre Institución_C.F.T. DIEGO PORTALES,Nombre Institución_C.F.T. JOHN F. KENNEDY,Nombre Institución_C.F.T. SANTO TOMÁS,Nombre Institución_C.F.T. SIMÓN BOLIVAR,Nombre Institución_ESCUELA DE AVIACIÓN,Nombre Institución_I.P. AIEP,Nombre Institución_I.P. DEL VALLE CENTRAL,Nombre Institución_I.P. DIEGO PORTALES,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX,Nombre Institución_I.P. LOS LEONES,Nombre Institución_I.P. SANTO TOMÁS,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE,Nombre Institución_PONTIFICIA U. CATÓLICA DE VALPARAÍSO,Nombre Institución_U. ACADEMIA DE HUMANISMO CRISTIANO,Nombre Institución_U. ADOLFO IBÁÑEZ,Nombre Institución_U. ADVENTISTA DE CHILE,Nombre Institución_U. ALBERTO HURTADO,Nombre Institución_U. ANDRÉS BELLO,Nombre Institución_U. ARTURO PRAT,Nombre Institución_U. AUSTRAL DE CHILE,Nombre Institución_U. AUTÓNOMA DE CHILE,Nombre Institución_U. BERNARDO O`HIGGINS,Nombre Institución_U. BOLIVARIANA,Nombre Institución_U. CATÓLICA CARDENAL RAÚL SILVA HENRÍQUEZ,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN,Nombre Institución_U. CATÓLICA DE TEMUCO,Nombre Institución_U. CATÓLICA DEL MAULE,Nombre Institución_U. CATÓLICA DEL NORTE,Nombre Institución_U. CENTRAL DE CHILE,Nombre Institución_U. CHILENO-BRITÁNICA DE CULTURA,Nombre Institución_U. DE ANTOFAGASTA,Nombre Institución_U. DE ARTE Y CIENCIAS SOCIALES ARCIS,Nombre Institución_U. DE ATACAMA,Nombre Institución_U. DE AYSEN,Nombre Institución_U. DE CHILE,Nombre Institución_U. DE CONCEPCIÓN,Nombre Institución_U. DE LA FRONTERA,Nombre Institución_U. DE LA SERENA,Nombre Institución_U. DE LAS AMÉRICAS,Nombre Institución_U. DE LOS ANDES,Nombre Institución_U. DE LOS LAGOS,Nombre Institución_U. DE MAGALLANES,Nombre Institución_U. DE O`HIGGINS,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. DE SANTIAGO DE CHILE,Nombre Institución_U. DE TALCA,Nombre Institución_U. DE TARAPACÁ,Nombre Institución_U. DE VALPARAÍSO,Nombre Institución_U. DE VIÑA DEL MAR,Nombre Institución_U. DEL ALBA,Nombre Institución_U. DEL BÍO-BÍO,Nombre Institución_U. DEL DESARROLLO,Nombre Institución_U. DEL MAR,Nombre Institución_U. DEL PACÍFICO,Nombre Institución_U. DIEGO PORTALES,Nombre Institución_U. FINIS TERRAE,Nombre Institución_U. GABRIELA MISTRAL,Nombre Institución_U. IBEROAMERICANA DE CIENCIAS Y TECNOLOGÍA,Nombre Institución_U. LA REPÚBLICA,Nombre Institución_U. MARÍTIMA DE CHILE,Nombre Institución_U. MAYOR,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. SAN SEBASTIÁN,Nombre Institución_U. SANTO TOMÁS,Nombre Institución_U. SEK,Nombre Institución_U. TECNOLÓGICA METROPOLITANA,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA,Nombre Institución_U. UCINF,Nombre Institución_infrequent_sklearn,Comuna donde se imparte la carrera o programa_Antofagasta,Comuna donde se imparte la carrera o programa_Arica,Comuna donde se imparte la carrera o programa_Calama,Comuna donde se imparte la carrera o programa_Chillán,Comuna donde se imparte la carrera o programa_Concepción,Comuna donde se imparte la carrera o programa_Copiapó,Comuna donde se imparte la carrera o programa_Coquimbo,Comuna donde se imparte la carrera o programa_Coyhaique,Comuna donde se imparte la carrera o programa_Curicó,Comuna donde se imparte la carrera o programa_El Bosque,Comuna donde se imparte la carre

In [10]:
print(f"Total number of columns: {inmuebles.shape[1] + labs.shape[1] + docentes.shape[1]}")

Total number of columns: 32


In [11]:
MERGE_COLS = [
    "Año",
    "Cód. Sede",
]

In [12]:
df_to_merge = docentes

X_values = df[MERGE_COLS].to_records(index=False).tolist()
Y_values = df_to_merge[MERGE_COLS].to_records(index=False).tolist()

X = set(X_values)
Y = set(Y_values)

In [13]:
len(X), len(Y), len(X & Y), len(X - Y), len(Y - X)

(2220, 5845, 1885, 335, 3960)

In [14]:
X_tuples = df[MERGE_COLS].apply(lambda x: tuple(x), axis=1)
notna_perc = X_tuples[X_tuples.isin(Y)].shape[0] / X_tuples.shape[0]
print(f"Percentage of not null values: {notna_perc * 100:.2f}%")

Percentage of not null values: 87.95%


## Merging Data

In [15]:
def merge_dataframes(
    target_df: pd.DataFrame,
    df_to_merge: pd.DataFrame,
    merge_cols: List[str],
) -> pd.DataFrame:
    """
    Merge two dataframes on the specified columns,
    conserving the index of the target dataframe
    Args:
        target_df (pd.DataFrame): dataframe to merge with
        df_to_merge (pd.DataFrame): dataframe to merge
        merge_cols (List[str]): columns to merge on
    Returns:
        pd.DataFrame: merged dataframe
    """
    target_df = target_df.copy()
    
    target_df["index"] = target_df.index
    df_merged = target_df.merge(df_to_merge, on=merge_cols, how="inner")
    df_merged = df_merged.set_index("index")
    df_merged.index.name = None
    return df_merged

In [16]:
df_merged = merge_dataframes(df, inmuebles, MERGE_COLS)
df_merged = merge_dataframes(df_merged, labs, MERGE_COLS)
df_merged = merge_dataframes(df_merged, docentes, MERGE_COLS)

In [17]:
show(df_merged)

(30607, 187)


,Año,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,Tipo Programa,Tipo Carrera,IngresoDirecto,Año Inicio Actividades,Duración (en semestres),Cód. Sede,Promedio Puntaje (promedio matemáticas y lenguaje),Puntaje de corte (primer seleccionado),Puntaje de corte (promedio de la carrera),Puntaje de corte (último seleccionado),Nº Alumnos Ingreso Via PSU o PDT,Valor de matrícula,Valor de arancel,Valor del Título,Vacantes,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula Total,Nombre Institución_C.F.T. DIEGO PORTALES,Nombre Institución_C.F.T. JOHN F. KENNEDY,Nombre Institución_C.F.T. SANTO TOMÁS,Nombre Institución_C.F.T. SIMÓN BOLIVAR,Nombre Institución_ESCUELA DE AVIACIÓN,Nombre Institución_I.P. AIEP,Nombre Institución_I.P. DEL VALLE CENTRAL,Nombre Institución_I.P. DIEGO PORTALES,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX,Nombre Institución_I.P. LOS LEONES,Nombre Institución_I.P. SANTO TOMÁS,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE,Nombre Institución_PONTIFICIA U. CATÓLICA DE VALPARAÍSO,Nombre Institución_U. ACADEMIA DE HUMANISMO CRISTIANO,Nombre Institución_U. ADOLFO IBÁÑEZ,Nombre Institución_U. ADVENTISTA DE CHILE,Nombre Institución_U. ALBERTO HURTADO,Nombre Institución_U. ANDRÉS BELLO,Nombre Institución_U. ARTURO PRAT,Nombre Institución_U. AUSTRAL DE CHILE,Nombre Institución_U. AUTÓNOMA DE CHILE,Nombre Institución_U. BERNARDO O`HIGGINS,Nombre Institución_U. BOLIVARIANA,Nombre Institución_U. CATÓLICA CARDENAL RAÚL SILVA HENRÍQUEZ,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN,Nombre Institución_U. CATÓLICA DE TEMUCO,Nombre Institución_U. CATÓLICA DEL MAULE,Nombre Institución_U. CATÓLICA DEL NORTE,Nombre Institución_U. CENTRAL DE CHILE,Nombre Institución_U. CHILENO-BRITÁNICA DE CULTURA,Nombre Institución_U. DE ANTOFAGASTA,Nombre Institución_U. DE ARTE Y CIENCIAS SOCIALES ARCIS,Nombre Institución_U. DE ATACAMA,Nombre Institución_U. DE AYSEN,Nombre Institución_U. DE CHILE,Nombre Institución_U. DE CONCEPCIÓN,Nombre Institución_U. DE LA FRONTERA,Nombre Institución_U. DE LA SERENA,Nombre Institución_U. DE LAS AMÉRICAS,Nombre Institución_U. DE LOS ANDES,Nombre Institución_U. DE LOS LAGOS,Nombre Institución_U. DE MAGALLANES,Nombre Institución_U. DE O`HIGGINS,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. DE SANTIAGO DE CHILE,Nombre Institución_U. DE TALCA,Nombre Institución_U. DE TARAPACÁ,Nombre Institución_U. DE VALPARAÍSO,Nombre Institución_U. DE VIÑA DEL MAR,Nombre Institución_U. DEL ALBA,Nombre Institución_U. DEL BÍO-BÍO,Nombre Institución_U. DEL DESARROLLO,Nombre Institución_U. DEL MAR,Nombre Institución_U. DEL PACÍFICO,Nombre Institución_U. DIEGO PORTALES,Nombre Institución_U. FINIS TERRAE,Nombre Institución_U. GABRIELA MISTRAL,Nombre Institución_U. IBEROAMERICANA DE CIENCIAS Y TECNOLOGÍA,Nombre Institución_U. LA REPÚBLICA,Nombre Institución_U. MARÍTIMA DE CHILE,Nombre Institución_U. MAYOR,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. SAN SEBASTIÁN,Nombre Institución_U. SANTO TOMÁS,Nombre Institución_U. SEK,Nombre Institución_U. TECNOLÓGICA METROPOLITANA,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA,Nombre Institución_U. UCINF,Nombre Institución_infrequent_sklearn,Comuna donde se imparte la carrera o programa_Antofagasta,Comuna donde se imparte la carrera o programa_Arica,Comuna donde se imparte la carrera o programa_Calama,Comuna donde se imparte la carrera o programa_Chillán,Comuna donde se imparte la carrera o programa_Concepción,Comuna donde se imparte la carrera o programa_Copiapó,Comuna donde se imparte la carrera o programa_Coquimbo,Comuna donde se imparte la carrera o programa_Coyhaique,Comuna donde se imparte la carrera o programa_Curicó,Comuna donde se imparte la carrera o programa_El Bosque,Comuna donde se imparte la carre

In [23]:
docentes

,Cód. Institución,Cód. Sede,Tipo Institución,N°DocentesJornadaMedia,N°DocentesJornadaHora,N°DocentesJornadaCompleta,N°HorasJornadaCompleta,N°HorasJornadaMedia,N°HorasJornadaHora,N°HorasProfJornadaHora,N°HorasProfJornadaMedia,N°HorasProfJornadaCompleta,N°ProfesionalJornadaHora,N°ProfesionalJornadaMedia,N°ProfesionalJornadaCompleta,N°Docentes,N°Horas,N°DocentesHombres,N°DocentesMujeres,Nombre Institución_C.F.T. ANDRÉS BELLO,Nombre Institución_C.F.T. CEDUC - UCN,Nombre Institución_C.F.T. DE TARAPACÁ,Nombre Institución_C.F.T. DIEGO PORTALES,Nombre Institución_C.F.T. INACAP,Nombre Institución_C.F.T. LOS LAGOS,Nombre Institución_C.F.T. PROANDES,Nombre Institución_C.F.T. PUCV,Nombre Institución_C.F.T. SALESIANOS DON BOSCO,Nombre Institución_C.F.T. SAN AGUSTÍN DE TALCA,Nombre Institución_C.F.T. SANTO TOMÁS,Nombre Institución_C.F.T. TEODORO WICKEL,Nombre Institución_I.P. AIEP,Nombre Institución_I.P. DE ARTES Y COMUNICACIÓN ARCOS,Nombre Institución_I.P. DE CHILE,Nombre Institución_I.P. DE LOS LAGOS,Nombre Institución_I.P. DEL VALLE CENTRAL,Nombre Institución_I.P. DIEGO PORTALES,Nombre Institución_I.P. DUOC UC,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX,Nombre Institución_I.P. INACAP,Nombre Institución_I.P. INTERNACIONAL DE ARTES CULINARIAS Y SERVICIOS,Nombre Institución_I.P. IPEGE,Nombre Institución_I.P. LA ARAUCANA,Nombre Institución_I.P. LATINOAMERICANO DE COMERCIO EXTERIOR,Nombre Institución_I.P. LOS LEONES,Nombre Institución_I.P. PROVIDENCIA,Nombre Institución_I.P. SANTO TOMÁS,Nombre Institución_I.P. VIRGINIO GÓMEZ,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE,Nombre Institución_U. ADOLFO IBÁÑEZ,Nombre Institución_U. ANDRÉS BELLO,Nombre Institución_U. ARTURO PRAT,Nombre Institución_U. AUSTRAL DE CHILE,Nombre Institución_U. AUTÓNOMA DE CHILE,Nombre Institución_U. BOLIVARIANA,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN,Nombre Institución_U. CATÓLICA DEL MAULE,Nombre Institución_U. CATÓLICA DEL NORTE,Nombre Institución_U. CENTRAL DE CHILE,Nombre Institución_U. DE ACONCAGUA,Nombre Institución_U. DE CONCEPCIÓN,Nombre Institución_U. DE LA FRONTERA,Nombre Institución_U. DE LA SERENA,Nombre Institución_U. DE LAS AMÉRICAS,Nombre Institución_U. DE LOS LAGOS,Nombre Institución_U. DE MAGALLANES,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. DE TALCA,Nombre Institución_U. DE TARAPACÁ,Nombre Institución_U. DE VALPARAÍSO,Nombre Institución_U. DE VIÑA DEL MAR,Nombre Institución_U. DEL ALBA,Nombre Institución_U. DEL BÍO-BÍO,Nombre Institución_U. DEL DESARROLLO,Nombre Institución_U. DEL MAR,Nombre Institución_U. DEL PACÍFICO,Nombre Institución_U. GABRIELA MISTRAL,Nombre Institución_U. LA REPÚBLICA,Nombre Institución_U. MAYOR,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. SAN SEBASTIÁN,Nombre Institución_U. SANTO TOMÁS,Nombre Institución_U. TECNOLÓGICA METROPOLITANA,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA,Nombre Institución_infrequent_sklearn,Sede_Angol,Sede_Antofagasta,Sede_Arica,Sede_Bellavista,Sede_Calama,Sede_Cañete,Sede_Chillán,Sede_Concepción,Sede_Constitución,Sede_Copiapó,Sede_Coquimbo,Sede_Coyhaique,Sede_Curicó,Sede_Iquique,Sede_La Serena,Sede_Las Condes,Sede_Linares,Sede_Los Andes,Sede_Los Ángeles,Sede_Melipilla,Sede_Osorno,Sede_Ovalle,Sede_Providencia,Sede_Puerto Montt,Sede_Punta Arenas,Sede_Quillota,Sede_Rancagua,Sede_San Antonio,Sede_San Felipe,Sede_San Fernando,Sede_Santiago,Sede_Talca,Sede_Talcahuano,Sede_Temuco,Sede_Valdivia,Sede_Valparaíso,Sede_Villarrica,Sede_Vitacura,Sede_Viña del Mar,Sede_infrequent_sklearn,Año
0,1001.0,1001001,0.0,680.0,1584.0,1727.0,75316.0,15040.0,14201.0,4117.0,4299.0,8129.0,464.0,194.0,190.0,3991.0,104557.0,2440.0,1551.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0